###### Please once look at it and understand
https://www.kaggle.com/granjithkumar/nlp-with-women-clothing-reviews

help for data cleaning
https://www.kaggle.com/suyashpratapsingh/eda-and-sentiment-analysis

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/hanzhang0420/Women-Clothing-E-commerce/master/Womens%20Clothing%20E-Commerce%20Reviews.csv')
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               23486 non-null  int64 
 1   Clothing ID              23486 non-null  int64 
 2   Age                      23486 non-null  int64 
 3   Title                    19676 non-null  object
 4   Review Text              22641 non-null  object
 5   Rating                   23486 non-null  int64 
 6   Recommended IND          23486 non-null  int64 
 7   Positive Feedback Count  23486 non-null  int64 
 8   Division Name            23472 non-null  object
 9   Department Name          23472 non-null  object
 10  Class Name               23472 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


In [ ]:
df = df.dropna(axis=0, subset=['Review Text']) #null review rows are dropped

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22641 entries, 0 to 23485
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               22641 non-null  int64 
 1   Clothing ID              22641 non-null  int64 
 2   Age                      22641 non-null  int64 
 3   Title                    19675 non-null  object
 4   Review Text              22641 non-null  object
 5   Rating                   22641 non-null  int64 
 6   Recommended IND          22641 non-null  int64 
 7   Positive Feedback Count  22641 non-null  int64 
 8   Division Name            22628 non-null  object
 9   Department Name          22628 non-null  object
 10  Class Name               22628 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.1+ MB


### Clean text

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
def tokens(words):
    words = re.sub("[^a-zA-Z]"," ", words)
    text = words.lower().split()                   
    return " ".join(text)
df['Review_clear'] = df['Review Text'].apply(tokens)
df.head()

In [ ]:
df['Review_clear'] = df['Review_clear'].astype(str)
stop_words = stopwords.words('english')
#clothing stopwords
clothes =['dress','color','wear','top','sweater','material','shirt','jeans','pant',
          'skirt','order','white','black','fabric','blouse','sleeve','even', 'jacket']

In [ ]:
def stopwords(review):
    text = [word.lower() for word in review.split() if word.lower() not in stop_words and word.lower() not in clothes]
    return " ".join(text)

In [ ]:
df['Review_clear'] = df['Review_clear'].apply(stopwords)
df.head()

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
lem = WordNetLemmatizer()

def lemma(text):
    lem_text = [lem.lemmatize(word) for word in text.split()]
    return " ".join(lem_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
df['Review_clear'] = df['Review_clear'].apply(lemma)
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
X = df['Review_clear']
ylabels = df['Recommended IND']
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
# from sklearn.ensemble import StackingClassifier 

from sklearn.linear_model import LogisticRegression

classifier3 = MLPClassifier(hidden_layer_sizes=(32,64), learning_rate="adaptive", max_iter=30,verbose=10, activation='relu', random_state=1, solver='adam')
classifier2 =  LinearSVC(C=0.01, dual=False, loss='squared_hinge', penalty='l2', tol=0.0001, random_state=42)
classifier1 = LogisticRegression()
models = [classifier1, classifier2, classifier3]

f_model = LogisticRegression()


In [ ]:
from mlxtend.classifier import StackingClassifier
classifier = StackingClassifier(classifiers = models, meta_classifier = f_model)


In [ ]:
vect = TfidfVectorizer(ngram_range=(1,2))
vect.fit(X_train, y_train)
# (vect.idf_).shape

In [ ]:
transf = TfidfTransformer()
transf.fit_transform(X_train,y_train)
(transf.id_).shape

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import LinearSVC
# from sklearn.naive_bayes import MultinomialNB
# classifier = MultinomialNB()
# classifier = LogisticRegression()
# classifier = SVC(kernel='linear')
# classifier =  LinearSVC(C=0.01, dual=False, loss='squared_hinge', penalty='l2', tol=0.0001)
# Create pipeline using Bag of Words
pipe = Pipeline([("vectorizer", TfidfVectorizer(ngram_range=(1,2))),
                 ('transformer', TfidfTransformer()),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)
# The combination of penalty='l2' and loss='hinge' are not supported when dual=False

Iteration 1, loss = 0.50189010
Iteration 2, loss = 0.26201187
Iteration 3, loss = 0.05033275
Iteration 4, loss = 0.00649542
Iteration 5, loss = 0.00292691
Iteration 6, loss = 0.00194116
Iteration 7, loss = 0.00150469
Iteration 8, loss = 0.00126880
Iteration 9, loss = 0.00112162
Iteration 10, loss = 0.00102072
Iteration 11, loss = 0.00094728
Iteration 12, loss = 0.00089081
Iteration 13, loss = 0.00084572
Iteration 14, loss = 0.00080821
Iteration 15, loss = 0.00077624
Iteration 16, loss = 0.00074841
Iteration 17, loss = 0.00072356
Iteration 18, loss = 0.00070103
Iteration 19, loss = 0.00068031
Iteration 20, loss = 0.00066102
Iteration 21, loss = 0.00064291
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_patt...
                                    meta_classifier=LogisticRegression(C=1.0,
                                                                       class_weight=None,
                                                                       dual=False,
                                                   

In [ ]:

# pipe['transformer'] shape
print(pipe['transformer'].idf_)
(pipe['transformer'].idf_).shape

In [ ]:
#Predicting class
predicted = pipe.predict(X_test)
predicted

array([1, 1, 0, ..., 1, 1, 1])

In [ ]:
#using mlp alone
#Confusion matrix 
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, classification_report

cm = confusion_matrix(y_test, predicted)
print(cm)
score = f1_score(y_test, predicted)
print(score*100)
print("Accuracy:",accuracy_score(y_test, predicted)*100)
print('Classification report is ')
print(classification_report(y_test, predicted))


[[ 459  362]
 [ 126 3582]]
93.62258233141662
Accuracy: 89.22499448001766
Classification report is 
              precision    recall  f1-score   support

           0       0.78      0.56      0.65       821
           1       0.91      0.97      0.94      3708

    accuracy                           0.89      4529
   macro avg       0.85      0.76      0.79      4529
weighted avg       0.89      0.89      0.88      4529



#stcked with 3
[[ 459  362]
 [ 126 3582]]
93.62258233141662
Accuracy: 89.22499448001766
Classification report is 
              precision    recall  f1-score   support

           0       0.78      0.56      0.65       821
           1       0.91      0.97      0.94      3708

    accuracy                           0.89      4529
   macro avg       0.85      0.76      0.79      4529
weighted avg       0.89      0.89      0.88      4529


# Stcked classi with 2 classifiers
[[ 440  401]
 [ 126 3562]]
93.11201150176448
Accuracy: 88.36387723559285
Classification report is 
              precision    recall  f1-score   support

           0       0.78      0.52      0.63       841
           1       0.90      0.97      0.93      3688

    accuracy                           0.88      4529
   macro avg       0.84      0.74      0.78      4529
weighted avg       0.88      0.88      0.87      4529


# mlp alone
[[ 456  372]
 [ 135 3566]]
93.36300562900904
Accuracy: 88.80547582247736
Classification report is 
              precision    recall  f1-score   support

           0       0.77      0.55      0.64       828
           1       0.91      0.96      0.93      3701

    accuracy                           0.89      4529
   macro avg       0.84      0.76      0.79      4529
weighted avg       0.88      0.89      0.88      4529


## Diff Classifiers
1. Logistic Regression
>> [[ 485  378]
>>[ 189 3477]]
>> f1_score: 92.46110889509374

2. SVM Classifier
>> [[ 514  349]
>> [ 250 3416]]
>> f1_score: 91.93917373166465

3. MultinomialNB
>> [[ 132  700]
>>  [ 632 3065]]
>> 82.14955775931386
>> Accuracy: 70.58953411349084

## pair of words
n_gram = (1,2)

1. MultinomialNB
>>[[ 137  692][  16 3684]]
  91.2332838038633
   Accuracy: 84.36741002428792
2. SVM Classifier
>> [[ 521  308]
 [ 210 3490]]
 93.0914910642838
 Accuracy: 88.56259659969088

3. Logistic Regression
>> [[ 512  317]
   [ 157 3543]]
   93.73015873015873
   Accuracy: 89.53411349083683

In [ ]:
pip install tpot

In [ ]:
from tpot import TPOTClassifier
from sklearn.metrics import roc_auc_score

# Instantiate TPOTClassifier
classifier = TPOTClassifier(
    generations=5,
    population_size=20,
    verbosity=2,
    scoring='roc_auc',
    random_state=42,
    disable_update_check=True,
    config_dict='TPOT sparse'
)
# tpot.fit(X_train, y_train)


In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from sklearn.naive_bayes import MultinomialNB
# classifier = MultinomialNB()
# classifier = LogisticRegression()
# classifier = SVC(kernel='linear')

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                  ('classifier', classifier) ])

# model generation
pipe.fit(X_train,y_train)

In [ ]:
print('\nBest pipeline steps:', end='\n')
for idx, (name, transform) in enumerate(pipe.fitted_pipeline_.steps, start=1):
    # Print idx and transform
    print(f'{idx}. {transform}')

# Best pipeline found out using TPOT
## Best pipeline: LinearSVC(input_matrix, C=0.01, dual=False, loss=squared_hinge, penalty=l2, tol=0.0001)
Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x7fc97bd0bf90>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 t...
                 TPOTClassifier(config_dict='TPOT sparse', crossover_rate=0.1,
                                cv=5, disable_update_check=True,
                                early_stop=None, generations=5, log_file=None,
                                max_eval_time_mins=5, max_time_mins=None,
                                memory=None, mutation_rate=0.9, n_jobs=1,
                                offspring_size=None,
                                periodic_checkpoint_folder=None,
                                population_size=20, random_state=42,
                                scoring='roc_auc', subsample=1.0, template=None,
                                use_dask=False, verbosity=2,
                                warm_start=False))],
         verbose=False)